# Regional Analysis

Analyse per gemeente voor halsbandparkiet en boomklever op een interactieve kaart.

In de notebook moet de soort, het jaartal en het gewenste parameter worden ingegeven.

## Conclusie 
Aantal unieke observators / km2 per gemeente in 2024

We verkiezen te werken met het aantal unieke observators per km2 per gemeente, zodat 1 persoon de data niet kan bepalen. </br>
De conclusie is dat de boomklever veel meer wijd verspreid is dan de halsbandparkiet.</br>
De halsbandparkiet bevindt zich vooral rond Brussel, Antwerpen en Kortrijk. Waarschijnlijk omdat rondom steden meer eten te vinden is, en de halsbandparkieten afhankelijk zijn van bijvoederen. En omdat ze profiteren van het hitte-effect van de stad. Halsbandparkieten verkiezen dan ook stadsparken als habitat.</br>
De boomklever bevindt zich zowat in het hele land, met uitzondering van een gedeelte van de kust en een regio in Luik.

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


In [37]:
df_yearly_observations_by_municipality = pd.read_parquet(f'../../../3_transformation/gold/yearly_observations_by_municipality.parquet', engine="pyarrow")

## Geodataframe gemeenten in belgië
belgium = gpd.read_file('../../../resources/geodata/kontur_boundaries_BE_20230628.gpkg')
belgium_gemeenten = belgium[(belgium["osm_admin_level"] == '8')].copy()
print(belgium.crs)


EPSG:4326


In [38]:
# Voeg geometrie terug toe vanuit het oorspronkelijke gemeenten-GeoDataFrame
df_yearly_merged_geometry = df_yearly_observations_by_municipality.merge(belgium_gemeenten[['name', 'geometry']], on='name', how='left')

# Maak opnieuw een geodataframe
gdf_yearly_observations_by_municipality = gpd.GeoDataFrame(df_yearly_merged_geometry, geometry='geometry', crs="EPSG:4326")
gdf_yearly_observations_by_municipality.head(5)

,name,year,observation_count_hp,observers_count_hp,observations_pym_hp,observations_pym_5yr_avg_hp,observations_growth_5yr_%_hp,observers_per_km2_hp,observations_per_km2_hp,observation_count_bk,observers_count_bk,observations_pym_bk,observations_pym_5yr_avg_bk,observations_growth_5yr_%_bk,observers_per_km2_bk,observations_per_km2_bk,area_km2,allbirds_observation_count,geometry
0,Aalst,1971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,2242,"MULTIPOLYGON (((3.98245 50.91942, 3.98282 50.9..."
1,Aalst,1972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,5281,"MULTIPOLYGON (((3.98245 50.91942, 3.98282 50.9..."
2,Aalst,1973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,6547,"MULTIPOLYGON (((3.98245 50.91942, 3.98282 50.9..."
3,Aalst,1974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,9548,"MULTIPOLYGON (((3.98245 50.91942, 3.98282 50.9..."
4,Aalst,1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.820668,9115,"MULTIPOLYGON (((3.98245 50.91942, 3.98282 50.9..."


In [39]:
species = { 
        'halsbandparkiet' : {'afkorting' : 'hp', 'titel' : 'halsbandparkiet'}, 
        'boomklever' : {'afkorting' : 'bk', 'titel' : 'boomklever'}
        }
field = { 
        'observations_scaled' : {'titel': 'Aantal Waarnemingen (per miljoen)', 'field': 'observations_pym_'},
        'observations_absolute' : {'titel': 'Aantal Waarnemingen', 'field': 'observation_count_'},
        'observators_unique' : {'titel': 'Unieke waarnemers', 'field': 'observers_count_'},
        'observators_unique_km2' : {'titel': 'Unieke waarnemers per km2', 'field': 'observers_per_km2_'}
        }

### Parameters ingeven voor het plotten van de kaart

In [40]:
species_selected = 'boomklever' # halsbandparkiet / boomklever
field_to_plot = "observators_unique_km2" # observations_absolute, observators_unique
to_plot_year = 2024

In [41]:
to_plot_field = field[field_to_plot]['field'] + species[species_selected]['afkorting']
map_title = f"{field[field_to_plot]['titel']} van de {species[species_selected]['titel']} per gemeente in {to_plot_year}"

gdf_observations_by_municipality_for_year = gdf_yearly_observations_by_municipality[gdf_yearly_observations_by_municipality["year"] == to_plot_year].copy()

gdf_observations_by_municipality_for_year["obs_km2"]= gdf_observations_by_municipality_for_year[to_plot_field]/gdf_observations_by_municipality_for_year["area_km2"]

# Voorkom problemen met log(0)
gdf_observations_by_municipality_for_year["obs_km2"] = gdf_observations_by_municipality_for_year["obs_km2"].replace(0, np.nan)  # Vermijd log(0)
gdf_observations_by_municipality_for_year["obs_km2"] = gdf_observations_by_municipality_for_year["obs_km2"].fillna(gdf_observations_by_municipality_for_year["obs_km2"].min() / 100)

# Zet naar EPSG:4326 (Folium vereist WGS84-coördinaten)
gdf_observations_by_municipality_for_year = gdf_observations_by_municipality_for_year.to_crs(epsg=4326)

# Define colormap and normalization
cmap = cm.plasma  # Standard colormap
norm = colors.LogNorm(vmin=gdf_observations_by_municipality_for_year["obs_km2"].min(), vmax=gdf_observations_by_municipality_for_year["obs_km2"].max())

# Function to get color from colormap
def get_color(value):
    return colors.rgb2hex(cmap(norm(value)))

# Maak een folium kaart
title_html = f'''
    <h1 style="
        position:absolute;
        z-index:100000;
        left:40vw;
        background-color:white;
        padding:10px;
        border-radius:5px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
        {map_title}
    </h1>
'''
m = folium.Map(location=[50.85, 4.35], zoom_start=9)  # Midden België

# Voeg gemeenten toe met interactieve hover
for _, row in gdf_observations_by_municipality_for_year.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, row=row: {
            "fillColor": get_color(row["obs_km2"]),
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.7,
        },
        tooltip=folium.Tooltip(f"{row['name']}: {row['obs_km2']:.3f} waarnemingen/km²"),
    ).add_to(m)
    
m.get_root().html.add_child(folium.Element(title_html))
m.save(f'plots/{map_title}.html')
# m -- do not show map in notebook to save space
